In [6]:
# !pip install boto3 aws-kinesis-agg
# !pip install kpl-py


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement kpl-py (from versions: none)
ERROR: No matching distribution found for kpl-py

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [7]:
# import boto3
# import time
# import json

# import aws_kinesis_agg.aggregator as aggregator

# # Set up Kinesis client
# kinesis_client = boto3.client('kinesis', region_name='us-east-1')

# # Kinesis Stream Name
# stream_name = 'brad-ml-training-stream'

# # Create an instance of the KinesisAggregator
# agg = aggregator.RecordAggregator()

# # Callback function to send the aggregated records to Kinesis Data Stream
# def send_to_kinesis_callback(encoded_data, partition_key):
#     kinesis_client.put_record(
#         StreamName=stream_name,
#         Data=encoded_data,
#         PartitionKey=partition_key
#     )

# # Assign the callback to the aggregator
# agg.on_record_complete(send_to_kinesis_callback)

# # Data to send
# data_to_send = [
#     {'sensor_id': 'sensor_1', 'temperature': 22.5, 'humidity': 30},
#     {'sensor_id': 'sensor_2', 'temperature': 23.0, 'humidity': 32},
#     {'sensor_id': 'sensor_3', 'temperature': 21.8, 'humidity': 29},
# ]

# # Loop over data and send to Kinesis stream using the KPL
# for record in data_to_send:
#     partition_key = record['sensor_id']  # Partition key can be sensor ID
#     json_data = json.dumps(record)       # Convert data to JSON
#     agg.add_user_record(partition_key, json_data)

# # Flush any remaining records


SyntaxError: invalid syntax (1803901089.py, line 45)

In [1]:
import boto3
import json
import uuid

from dotenv import load_dotenv
load_dotenv()


# Define the AWS region and Kinesis stream name
region_name = 'us-east-1'  # Update to your desired region
stream_name = 'brad-ml-training-stream'  # Replace with your stream name

# Initialize the boto3 Kinesis client
kinesis_client = boto3.client('kinesis', region_name=region_name)

# Function to send data to Kinesis using put_record
def send_data_to_kinesis(partition_key, data):
    try:
        response = kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),  # Convert data to JSON string
            PartitionKey=partition_key  # Partition key to determine the shard
        )
        print(f"Record sent. Sequence Number: {response['SequenceNumber']}")
    except Exception as e:
        print(f"Error sending data to Kinesis: {str(e)}")

# Example usage
partition_key = str(uuid.uuid4())
# partition_key = 'partition_key_1'
data = {
    'message': 'Hello, this is a test message',
    'timestamp': '2024-09-12T12:00:00Z',
    'value': 123
}

send_data_to_kinesis(partition_key, data)


Error sending data to Kinesis: An error occurred (UnrecognizedClientException) when calling the PutRecord operation: The security token included in the request is invalid


In [7]:
import boto3
import json
import uuid
import requests
from dotenv import load_dotenv

load_dotenv()


# Define the AWS region and Kinesis stream name
region_name = 'us-east-1'  # Update to your desired region
stream_name = 'brad-ml-training-stream'  # Replace with your stream name

# Initialize the boto3 Kinesis client
kinesis_client = boto3.client('kinesis', region_name=region_name)
push_to_kinesis = []



def get_sample_data():
    response = requests.get('https://randomuser.me/api/?results=600')
    data = response.json()
    sample_data = []
    # Access the data returned from the API
    results = data['results']
    for result in results:
        first_name = result['name']['first']
        last_name = result['name']['last']
        age = result['dob']['age']
        gender = result['gender']
        lattitude = result['location']['coordinates']['latitude']
        longitude = result['location']['coordinates']['longitude']
        sample_data.append({
            'first_name': first_name,
            'last_name': last_name,
            'age': age,
            'gender': gender
        })
    return sample_data

# Function to send data to Kinesis using put_record
def send_data_to_kinesis(stream_name, records):
    try:
        response = kinesis_client.put_records(
                StreamName=stream_name,
                Records=records,
            )
        print(f"Records sent. Sequence Number: {response}")
    except Exception as e:
        print(f"Error sending data to Kinesis: {str(e)}")


# test_records = [{'this': 'is', 'a': 'test'}, {'this': 'is', 'second': 'test'}]
test_records = get_sample_data()

record_count = 0
for record in test_records:
    push_to_kinesis.append(
        {'Data': json.dumps(record), 'PartitionKey': str(uuid.uuid4())}
    )
    record_count += 1
    if record_count >= 500:
        send_data_to_kinesis(stream_name, push_to_kinesis)
        record_count = 0

if record_count < 500:
    send_data_to_kinesis(stream_name, push_to_kinesis)
    record_count = 0
    push_to_kinesis = []



Records sent. Sequence Number: {'FailedRecordCount': 0, 'Records': [{'SequenceNumber': '49655911971634696746036724279747190520353312385477378050', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279748399446172927014652084226', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279749608371992541643826790402', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279750817297812156273001496578', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279752026223631770902176202754', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279753235149451385531350908930', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279754444075271000160525615106', 'ShardId': 'shardId-000000000000'}, {'SequenceNumber': '49655911971634696746036724279755653001090614789700321282', 'ShardId': 'shardId-000000000000'}, {'SequenceNu